In [5]:
import os
import requests
import logging
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.lib.colors import black, white
from reportlab.lib.colors import HexColor
import random


In [6]:
class MangaStoryGenerator:
    def __init__(self, phi3_token):
        
        self.logger = logging.getLogger(__name__)
        self.phi3_url = "https://cu-vertical-dimensional-continuity.trycloudflare.com/phi3/generate"
        self.phi3_token = phi3_token

    def generate_story_outline(self, prompt, max_chapters=3, max_tokens=750):
    
        headers = {
            "Authorization": f"Bearer {self.phi3_token}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "inputs":f"<|system|>\n You are a creative manga story writer generating a {max_chapters}- scene manga comic strips.\n"
                    "Include compelling dialogues."
                    "Ensure narrative flow and character development."
                    "Maintain consistent tone and progression."
                    "Focus on maintaining narrative continuity and dramatic progression.<|end|>\n"
                    f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
                    "parameters": {
                        "max_new_tokens": max_tokens,
                        "temperature": 0.7
                    }
        }
        
        try:
            response = requests.post(self.phi3_url, headers=headers, json=payload)
            response.raise_for_status()
            
            story_outline = response.json().get("generated_text", "").split("\n")
            story_outline = [line.strip() for line in story_outline if line.strip()]
            
            self.logger.info(f"Generated story outline with {len(story_outline)} scenes")
            return story_outline
        
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Story generation failed: {e}")
            return []

In [7]:
class MangaImageGenerator:
    def __init__(self, flux_token, output_dir="manga_output"):
        """
        Initialize Image Generator
        
        :param flux_token: Authentication token for image generation
        :param output_dir: Directory to store generated images
        """
        self.logger = logging.getLogger(__name__)
        self.flux_url = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
        self.flux_token = flux_token
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)

    def generate_scene_images(self, scenes):
        """
        Generate manga-style scene images for multiple scenes
        
        :param scenes: List of scene descriptions
        :return: List of image paths
        """
        images = []
        
        for scene in scenes:
            image_path = self.generate_single_scene_image(scene)
            if image_path:
                images.append(image_path)
        
        return images

    def generate_single_scene_image(self, scene_description):
        """
        Generate a single manga-style scene image
        
        :param scene_description: Textual description of the scene
        :return: Path to generated image
        """
        headers = {
            "Authorization": f"Bearer {self.flux_token}",
            "Content-Type": "application/json"
        }
        prompt = f"Cinematic manage comic strip 1-3 images/panel: {scene_description.replace('*', '')}."+ "Animeart style,dynamic composition, emotional storytelling, vibrant colors, detailed characters, sequential art flow."
        if len(prompt) > 200:
            prompt = prompt.replace(' ', '')
        payload = {
            "prompt": prompt[:200],
            "img_size": 768,
            "guidance_scale": 8.5,
            "num_inference_steps": 50,
            "negative_prompt": "low quality, blurry, sketch, draft"
        }
        
        try:
            print(scene_description)
            response = requests.post(self.flux_url, headers=headers, json=payload)
            response.raise_for_status()
            
            output_path = os.path.join(
                self.output_dir, 
                f"scene_{len(os.listdir(self.output_dir))}.png"
            )
            
            with open(output_path, "wb") as f:
                f.write(response.content)
            
            self.logger.info(f"Generated scene image: {output_path}")
            return output_path
        
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Image generation error: {e}")
            return None


In [9]:
import os
import logging
import random
import re

class MangaPDFGenerator:
    def __init__(self, output_dir="manga_output"):
        """
        Initialize PDF Generator with manga-specific layout configurations
        
        :param output_dir: Directory to store generated PDFs
        """
        self.logger = logging.getLogger(__name__)
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Predefined color palettes for backgrounds and bubbles
        self.background_colors = [
            '#F0F4F8', '#E6F2FF', '#E6FFFA', '#FFF5E6', 
            '#F0E6FF', '#FFEBEE', '#E8F5E9', '#FFF3E0'
        ]
        
        self.bubble_colors = [
            '#FFFFFF', '#F0F0F0', '#E6E6FA', '#F0FFFF', 
            '#FFF0F5', '#F5F5DC', '#F0FFF0', '#FAFAD2'
        ]
    def _get_random_color(self, color_palette):
        """
        Get a random color from the given palette
        
        :param color_palette: List of color hex codes
        :return: Randomly selected color
        """
        return random.choice(color_palette)
    def _preprocess_text(self, text):
        """
        Preprocess text to remove unwanted elements
        
        :param text: Input text
        :return: Cleaned text
        """
        # Remove text in parentheses or brackets
        text = re.sub(r'\[.*?\]|\(.*?\)', '', text)
        
        # Remove scene labels
        text = re.sub(r'^scene\s*\d+\s*:\s*', '', text, flags=re.IGNORECASE)
        
        # Strip leading/trailing whitespace
        text = text.strip()
        
        return text
    def _load_manga_fonts(self, c):
        """
        Register custom manga-style fonts
        
        :param c: Canvas object
        :return: Dictionary of registered fonts
        """
        try:
            # Register custom manga-style fonts
            pdfmetrics.registerFont(TTFont('MangaFont', './Fonts/mangat.ttf'))
            pdfmetrics.registerFont(TTFont('DialogueFont', './Fonts/Dialogue-A-Light-Italic.ttf'))
            
            return {
                'title': 'MangaFont',
                'dialogue': 'DialogueFont'
            }
        except Exception as e:
            self.logger.warning(f"Custom font registration failed: {e}")
            return {
                'title': 'Helvetica-Bold',
                'dialogue': 'Helvetica'
            }
    def _smart_wrap_text(self, text, max_width, font_name, font_size, max_lines=4):
        """
        Intelligently wrap text to fit within a specified width
        
        :param text: Input text
        :param max_width: Maximum width of the text area
        :param font_name: Name of the font
        :param font_size: Size of the font
        :param max_lines: Maximum number of lines allowed
        :return: List of wrapped text lines
        """
        words = text.split()
        lines = []
        current_line = []

        for word in words:
            # Check if adding this word would exceed max width
            test_line = ' '.join(current_line + [word])
            line_width = stringWidth(test_line, font_name, font_size)
            
            if line_width <= max_width:
                current_line.append(word)
            else:
                # If line is not empty, add current line and start a new one
                if current_line:
                    lines.append(' '.join(current_line))
                current_line = [word]

            # Stop if we've reached max lines
            if len(lines) >= max_lines:
                break

        # Add any remaining words to the last line
        if current_line and len(lines) < max_lines:
            lines.append(' '.join(current_line))

        # Truncate to max lines and add ellipsis if needed
        if len(lines) > max_lines:
            lines = lines[:max_lines]
            lines[-1] += '...'

        return lines
    def _create_scene_border(self, c, x, y, width, height, scene_bg_color=None):
        """
        Create a decorative border for a manga scene
        
        :param c: Canvas object
        :param x: X coordinate of scene
        :param y: Y coordinate of scene
        :param width: Scene width
        :param height: Scene height
        :param scene_bg_color: Background color for the scene
        """
        # Scene background
        if scene_bg_color:
            c.setFillColor(HexColor(scene_bg_color))
            c.rect(x, y, width, height, fill=1, stroke=0)
        
        # Manga-style border with rough, hand-drawn effect
        c.setStrokeColor(HexColor('#333333'))
        c.setLineWidth(1.5)
        
        # Slightly irregular border with padding
        border_padding = 5
        c.line(x + border_padding, y + border_padding, x + width - border_padding, y + border_padding)
        c.line(x + width - border_padding, y + border_padding, x + width - border_padding, y + height - border_padding)
        c.line(x + width - border_padding, y + height - border_padding, x + border_padding, y + height - border_padding)
        c.line(x + border_padding, y + height - border_padding, x + border_padding, y + border_padding)
    def _create_speech_bubble(self, c, x, y, text, max_width=200, padding=10, bg_color='#F0F0F0', is_narration=False):
        """
        Create a manga-style speech bubble or narration box
        
        :param c: Canvas object
        :param x: X coordinate
        :param y: Y coordinate
        :param text: Text content
        :param max_width: Maximum width of the bubble
        :param padding: Padding inside the bubble
        :param bg_color: Background color of the bubble
        :param is_narration: Flag to indicate narration style
        :return: Height of the created bubble
        """
        # Preprocess text
        text = self._preprocess_text(text)
        
        # Determine font based on narration status
        font_name = 'DialogueFont' if not is_narration else 'Helvetica-Oblique'
        font_size = 10
        
        # Wrap text to fit within bubble
        wrapped_text = self._smart_wrap_text(
            text, 
            max_width - 2*padding, 
            font_name, 
            font_size,
            max_lines=4  # Limit to 4 lines
        )
        
        # Calculate bubble dimensions
        text_height = len(wrapped_text) * 15
        bubble_width = max_width
        bubble_height = text_height + 2 * padding
        
        # Draw bubble with hand-drawn style
        c.setFillColor(HexColor(bg_color))
        c.setStrokeColor(black)
        
        # Slightly irregular bubble shape
        c.saveState()
        c.translate(x, y)
        c.rotate(random.uniform(-2, 2))  # Slight rotation for hand-drawn effect
        
        # Create different shapes for dialogue and narration
        path = c.beginPath()
        if is_narration:
            # Narration box with softer edges
            path.moveTo(10, 0)
            path.lineTo(bubble_width - 10, 0)
            path.curveTo(bubble_width, 0, bubble_width, 10, bubble_width, 20)
            path.lineTo(bubble_width, bubble_height - 10)
            path.curveTo(bubble_width, bubble_height, bubble_width - 10, bubble_height, bubble_width - 20, bubble_height)
            path.lineTo(10, bubble_height)
            path.curveTo(0, bubble_height, 0, bubble_height - 10, 0, bubble_height - 20)
            path.lineTo(0, 10)
            path.curveTo(0, 0, 10, 0, 20, 0)
        else:
            # Speech bubble with pointer
            path.moveTo(0, 0)
            path.lineTo(bubble_width, 0)
            path.lineTo(bubble_width, bubble_height)
            
            # Add a small pointer to simulate speech direction
            path.lineTo(bubble_width - 20, bubble_height)
            path.lineTo(bubble_width - 40, bubble_height + 20)
            path.lineTo(bubble_width - 60, bubble_height)
            path.lineTo(0, bubble_height)
        path.close()
        
        c.drawPath(path, fill=1, stroke=1)
        
        # Add text
        c.setFont(font_name, font_size)
        c.setFillColor(black)
        
        text_object = c.beginText(padding, padding)
        for line in wrapped_text:
            text_object.textLine(line)
        c.drawText(text_object)
        
        c.restoreState()
        
        return bubble_height + 2 * padding  # Total bubble height including padding

    def create_manga_pdf(self, images, texts, is_narration=None):
        """
        Create a professional manga-style PDF with single image per page
        
        :param images: List of image paths
        :param texts: List of corresponding texts (dialogue or narration)
        :param is_narration: Optional list of boolean flags to indicate narration
        :return: Path to generated PDF
        """
        # Validate input
        #if len(images) != len(texts):
            #raise ValueError("Number of images must match number of texts")
        
        # Default narration flags if not provided
        if is_narration is None:
            is_narration = [False] * len(texts)
        elif len(is_narration) != len(texts):
            raise ValueError("Number of narration flags must match number of texts")
        
        pdf_path = os.path.join(self.output_dir, "manga_series.pdf")
        c = canvas.Canvas(pdf_path, pagesize=letter)
        width, height = letter
        
        # Margins and padding
        margin_x = inch * 1
        margin_y = inch * 1
        
        # Image sizing (70% of page)
        image_width = width * 0.7
        image_height = height * 0.7
        
        # Image positioning (centered)
        image_x = (width - image_width) / 2
        image_y = (height - image_height - inch) / 2
        
        # Load fonts
        fonts = self._load_manga_fonts(c)
        
        # Iterate through images
        for page_num, (img_path, text, narration) in enumerate(zip(images, texts, is_narration)):
            # Page background
            c.setFillColor(HexColor('#F5F5F5'))
            c.rect(0, 0, width, height, fill=1, stroke=0)
            
            # Add chapter title
            c.setFont(fonts['title'], 16)
            c.drawString(margin_x, height - margin_y/2, f"Chapter {page_num + 1}")
            
            # Background color for scene
            scene_bg_color = self._get_random_color(self.background_colors)
            bubble_color = self._get_random_color(self.bubble_colors)
            
            # Create scene border
            self._create_scene_border(
                c, image_x, image_y, 
                image_width, image_height, 
                scene_bg_color
            )
            
            # Add image
            c.saveState()
            c.translate(image_x, image_y)
            c.rotate(random.uniform(-1, 1))
            c.drawImage(
                img_path, 
                0, 0, 
                width=image_width, 
                height=image_height
            )
            c.restoreState()
            
            # Add speech bubble
            bubble_height = self._create_speech_bubble(
                c, 
                x=image_x, 
                y=image_y - 1*inch, 
                text=text,
                max_width=image_width,
                bg_color=bubble_color,
                is_narration=narration
            )
            
            # Add page number
            c.setFont('Helvetica', 10)
            c.setFillColor(black)
            c.drawRightString(width - margin_x/2, margin_y/2, f"Page {page_num + 1}")
            
            c.showPage()

        c.save()
        self.logger.info(f"Manga PDF created at {pdf_path}")
        return pdf_path

In [10]:
class MangaGenerator:
    def __init__(self, phi3_token, flux_token, output_dir="manga_output"):
        """
        Comprehensive Manga Generation
        
        :param phi3_token: Token for story generation
        :param flux_token: Token for image generation
        :param output_dir: Output directory for manga assets
        """
        # Configure logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s: %(message)s'
        )
        
        self.story_generator = MangaStoryGenerator(phi3_token)
        self.image_generator = MangaImageGenerator(flux_token, output_dir)
        self.pdf_generator = MangaPDFGenerator(output_dir)

    def generate_manga(self, prompt, max_chapters=6):
        """
        Comprehensive manga generation workflow
        
        :param prompt: Story concept
        :param max_chapters: Number of chapters
        :return: Path to generated PDF
        """
        # Generate story outline
        story_outline = self.story_generator.generate_story_outline(prompt, max_chapters)
        
        if not story_outline:
            logging.error("No story outline generated. Exiting.")
            return None
        
        # Generate scene images
        scene_images = self.image_generator.generate_scene_images(story_outline)
        
        # Create manga PDF
        print(scene_images)
        print(story_outline)
        manga_pdf = self.pdf_generator.create_manga_pdf(scene_images, story_outline)
        
        return manga_pdf

In [13]:
def main():
    PHI3_TOKEN = "AEulafWZQ1xhIkeLgjcp5nhL7d0cLPWiXAU34DTVvXI"  # Replace with actual token
    FLUX_TOKEN = "calm-bold-tree-bd1b1c8bc141403231e7f0a8" 

    # Story prompt
    prompt = "In a postapocaluptic world, humans have in to a multi solar system species across the galaxy they fight each other and will they co-operate to defend againt gaint humanoid aliens" 
    # Initialize and generate manga
    manga_generator = MangaGenerator(PHI3_TOKEN, FLUX_TOKEN)
    manga_pdf = manga_generator.generate_manga(prompt)
    
    if manga_pdf:
        print(f"Manga PDF generated: {manga_pdf}")



In [ ]:
if __name__ == "__main__":
    main()

2024-12-18 22:54:24,516 - INFO: Generated story outline with 31 scenes


**Title:** "Harmony Amongst the Stars"


2024-12-18 22:54:52,174 - INFO: Generated scene image: manga_output/scene_0.png


**Characters:**


2024-12-18 22:55:19,764 - INFO: Generated scene image: manga_output/scene_1.png


...

**Scene 2: The Secret Alliance**


2024-12-18 23:02:12,622 - INFO: Generated scene image: manga_output/scene_16.png


*Setting: A concealed meeting location on an asteroid, with Lena, Kiro, and Zyra.*


2024-12-18 23:02:40,349 - INFO: Generated scene image: manga_output/scene_17.png


*Gantos, alarmed by their actions, sends his fleet to intercept them.*
